In [3]:
import requests
import json
import time
import getopt
import sys
import os
import pandas as pd


In [5]:
USERNAME = "rcs04@mail.aub.edu"
PASSWORD = "Iloveapplle123!!"
CLIENT_ID = "d383a5a913ea43739e4cd6fb50d8a26d4cee32da"


In [6]:
# Set Application Constants
RDP_AUTH_VERSION = "/v1"
RDP_FILINGS_VERSION = "/beta1"
RDP_BASE_URL = "https://api.refinitiv.com"
CATEGORY_URL = "/auth/oauth2"
ENDPOINT_URL = "/token"
CLIENT_SECRET = ""
TOKEN_FILE = "token.txt"
SCOPE = "trapi"


## Define Token Handling and Obtain a Valid Token

In [7]:
TOKEN_ENDPOINT = RDP_BASE_URL + CATEGORY_URL + RDP_AUTH_VERSION + ENDPOINT_URL


def _requestNewToken(refreshToken):
    if refreshToken is None:
        tData = {
            "username": USERNAME,
            "password": PASSWORD,
            "grant_type": "password",
            "scope": SCOPE,
            "takeExclusiveSignOnControl": "true"
        }
    else:
        tData = {
            "refresh_token": refreshToken,
            "grant_type": "refresh_token",
        }

    # Make a REST call to get latest access token
    response = requests.post(
        TOKEN_ENDPOINT,
        headers={
            "Accept": "application/json"
        },
        data=tData,
        auth=(
            CLIENT_ID,
            CLIENT_SECRET
        )
    )

    if response.status_code != 200:
        raise Exception(
            "Failed to get access token {0} - {1}".format(response.status_code, response.text))

    # Return the new token
    return json.loads(response.text)


def saveToken(tknObject):
    tf = open(TOKEN_FILE, "w+")
    print("Saving the new token")
    # Append the expiry time to token
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10
    # Store it in the file
    json.dump(tknObject, tf, indent=4)


def getToken():
    try:
        print("Reading the token from: " + TOKEN_FILE)
        # Read the token from a file
        tf = open(TOKEN_FILE, "r+")
        tknObject = json.load(tf)

        # Is access token valid
        if tknObject["expiry_tm"] > time.time():
            # return access token
            print(tknObject["expiry_tm"])
            print("time.time()=" + str(time.time()))
            return tknObject["access_token"]

        print("Token expired, refreshing a new one...")
        tf.close()
        # Get a new token from refresh token
        tknObject = _requestNewToken(tknObject["refresh_token"])

    except Exception as exp:
        print("Caught exception: " + str(exp))
        print("Getting a new token using Password Grant...")
        tknObject = _requestNewToken(None)

    # Persist this token for future queries
    saveToken(tknObject)
#    print("Token is: " + tknObject["access_token"])
    # Return access token
    return tknObject["access_token"]


accessToken = getToken()
print("Have token now")
print("Token is: " + accessToken)


Reading the token from: token.txt
Caught exception: [Errno 2] No such file or directory: 'token.txt'
Getting a new token using Password Grant...
Saving the new token
Have token now
Token is: eyJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImRMdFd2Q0tCSC1NclVyWm9YMXFod2pZQ2t1eDV0V2ZSS2o4ME9vcjdUY28ifQ.eyJkYXRhIjoie1wiY2lwaGVydGV4dFwiOlwidGltM0dfX0tRM3NPb1REeUdqLWpZcDJFTVBuM3RTZXlua3RlbEQ3RG9XZURGa2VHU2RuZ1ZaQzg0dkd1RkkxZ2UwbjRxUGRWSG9kUWF1c0YybzJUVlNFZllBRndsU1RUN3BVaTYxeklLSVhHdXlXajMyeVhvR2l3T3p6Y3F1aUZ2QnMwdXJrRzkyN3ltSVJCQWlHbVF5c0o5V2dVNll0TnZmbWdlLTlhOUlkdVc3X2gtRTZtQll0eTEzR2NjNUg0M1QzZ1Q2QnZRZzhzNHczeDhCalB0OTRWVkIteXRqM1hZUHJyd2VLRWFRMXNxcFFydmxvSEQ3YlVRZmR6Nnk1UVM4NXhkX2N4UHc4dE45ZDF5cWFFRWpyV2t0UWJpRFZMVGluOVFqM2tFQnZmMHpBMHFST2U3TktNWkdjMTlJMlZmdDVFZlpqSUpybWVVUlIyVlNKYlVwdlc2dE9iTjNjXzIwY1lZaTVtZC1yWkp1LUl6czcxT2FpXzB3dVEzZWZyS09idDIxRXFwbjBwSEZRY1dCdVd6MmtUbVR1cDliWkIydGo5MlVJalBkZE5HUElHdFFuS3ZLdy1UYlhlUDVGbWRrTkkxSExPa19xRnpMM3JER0NwQURZLXUxbk9iRnBwMXd0X2xOSWswN2RCLTBlRz

## Define Filings Helper Function requestSearch

In [8]:
FILINGS_ENDPOINT = RDP_BASE_URL+'/data-store'+RDP_FILINGS_VERSION + '/graphql'


def requestSearch(token, payloadSearch):

    global FILINGS_ENDPOINT
    print("requestSearch...")

    querystring = {}
    payload = ""
    jsonfull = ""
    jsonpartial = ""

    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer " + token,
        'cache-control': "no-cache"
    }

    response = requests.post(FILINGS_ENDPOINT, json={
                             'query': payloadSearch}, headers=headers)

    print("Response status code ="+str(response.status_code))

    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
            accessToken = getToken()     # token refresh on token expired
            headers["Authorization"] = "Bearer " + accessToken
            response = requests.post(FILINGS_ENDPOINT, json={
                'query': payloadSearch}, headers=headers)

    print('Raw response=')
    print(response)

    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        return jsonFullResp
    else:
        return ''


## Search Filings by File Type

In [9]:
# The following example searched for all 10-Qs on February 12, 2021
payloadIn = """
{
  FinancialFiling(filter: {AND: [
    {FilingDocument: {DocumentSummary: {FormType: {EQ: "10-K"}}}}, 
    {FilingDocument: {DocumentSummary: {FilingDate: {BETWN: {FROM: "2021-02-12T00:00:00Z", TO: "2021-02-12T23:59:59Z"}}}}}]}, 
    sort: {FilingDocument: {DocumentSummary: {FilingDate: DESC}}},
    limit: 25 ) {
    _metadata {
      totalCount
      cursor
    }
    FilingOrganization {
      Names {
        Name {
          OrganizationName (filter: {OrganizationNameTypeCode: {EQ: "LNG"}}){
            OrganizationName
          }
        }
      }
    }
    FilingDocument {
      Identifiers {
        OrganizationId
        Dcn
      }
      DocId
      FinancialFilingId
      DocumentSummary {
        DocumentTitle
        FeedName
        FormType
        HighLevelCategory
        MidLevelCategory
        FilingDate
        SecAccessionNumber
        SizeInBytes
      }
      FilesMetaData {
        FileName
        MimeType
      }
    }
  }
}
"""
jsonFullResp = requestSearch(accessToken, payloadIn)
print('Parsed json response=')
print(json.dumps(jsonFullResp, indent=2))
docId = jsonFullResp["data"]["FinancialFiling"][0]["FilingDocument"]["DocId"]
print('DocId is', str(docId))
cursor = jsonFullResp["data"]["FinancialFiling"][0]["_metadata"]["cursor"]
print('cursor is', str(cursor))


requestSearch...
Response status code =404
Raw response=
<Response [404]>
Parsed json response=
""


TypeError: string indices must be integers

## Search by OrganizationId

In [ ]:
# Search for all filings documents for Tesla in 2021

payloadIn = """
{
  FinancialFiling(filter: {AND: [{FilingDocument: {Identifiers: {OrganizationId: {EQ: "4297089638"}}}}, 
    {FilingDocument: {DocumentSummary: {FilingDate: {BETWN: {FROM: "2021-01-01T00:00:00Z", TO: "2021-12-31T11:59:59Z"}}}}}]}, 
    sort: {FilingDocument: {DocumentSummary: {FilingDate: DESC}}}, 
    limit: 10) {
    _metadata {
      totalCount
    }
    FilingOrganization {
      Names {
        Name {
          OrganizationName (filter: {OrganizationNameTypeCode: {EQ: "LNG"}}){
            OrganizationName
          }
        }
      }
    }
    FilingDocument {
      Identifiers {
        OrganizationId
        Dcn
      }
      DocId
      FinancialFilingId
      DocumentSummary {
        DocumentTitle
        FeedName
        FormType
        HighLevelCategory
        MidLevelCategory
        FilingDate
        SecAccessionNumber
        SizeInBytes
      }
      FilesMetaData {
        FileName
        MimeType
      }
    }
  }
}
"""
jsonFullResp = requestSearch(accessToken, payloadIn)
print('Parsed json response=')
print(json.dumps(jsonFullResp, indent=2))


## Download Filings Documents
There are four identifers or retrieval methods you can use to download a document.

* FilingId (FilingId, or Financial Filing Id, is an internal permanent identifier assigned to each filings document. This is our strategic filings identifier.)
* Dcn (Dcn, also known as Document Control Number, is an external identifier and an enclosed film-number specific to Edgar documents.)
* DocId (DocId, or Document Identifier, is an internal identifier assigned to financial filings documents.)
* Filename (Filename provides a faster and direct route to download documents without going through a resolver.)

### Define Helper Function retrieveURL

In [ ]:
def retrieveURL(token, retrievalParameters):

    ENDPOINT_DOC_RETRIEVAL = RDP_BASE_URL+'/data/filings' + \
        RDP_FILINGS_VERSION + '/retrieval/search/' + retrievalParameters

    headers = {
        "Authorization": "Bearer " + token,
        "X-API-Key": "155d9dbf-f0ac-46d9-8b77-f7f6dcd238f8",
        "ClientID": "api_playground"
    }
    print("Next we retrieve: " + ENDPOINT_DOC_RETRIEVAL)

    response = requests.get(ENDPOINT_DOC_RETRIEVAL, headers=headers)

    print("Response status code ="+str(response.status_code))

    if response.status_code != 200:
        if response.status_code == 401:   # error when token expired
            token = getToken()     # token refresh on token expired
            print("Token now is: "+token)
            headers["Authorization"] = "Bearer " + token
            response = requests.get(
                ENDPOINT_DOC_RETRIEVAL, headers=headers)

    print("Response status code ="+str(response.status_code))
    if response.status_code == 200:
        jsonFullResp = json.loads(response.text)
        return jsonFullResp
    else:
        return ''


### Retrieve URL by DocID

In [ ]:
jsonFullResp = retrieveURL(accessToken, 'docId/54932207')
print('full response is =')
print(json.dumps(jsonFullResp, indent=2))
fileName = list(jsonFullResp.keys())[0]
print("fileName is: ")
print(fileName)
signedUrl = jsonFullResp[list(jsonFullResp.keys())[0]]["signedUrl"]
print("signedUrl to retrieve is: ")
print(signedUrl)


### Retrieve URL by FilingID

In [ ]:
jsonFullResp = retrieveURL(accessToken, 'filingId/97661417885')
print('full response is =')
print(json.dumps(jsonFullResp, indent=2))
fileName = list(jsonFullResp.keys())[0]
print("fileName is: ")
print(fileName)
signedUrl = jsonFullResp[list(jsonFullResp.keys())[0]]["signedUrl"]
print("signedUrl to retrieve is: ")
print(signedUrl)


### Download the Document

In [ ]:
def retrieveSaveDoc(fileName, signedUrl, token):

    headers = {
        'clientId': CLIENT_ID,
        'Authorization': "Bearer " + token
    }

    response = requests.get(signedUrl, headers=headers, allow_redirects=True)

    if response.status_code == 200:
        filenameWithDir = './downloads/' + str(fileName)
        os.makedirs(os.path.dirname(filenameWithDir), exist_ok=True)
        open(filenameWithDir, 'wb').write(response.content)
        print("The document ", fileName,
              " has been downloaded indo downloads subfolder")
        return fileName
    else:
        print("Response code on error is:", str(response.status_code))
        return ''


retrieveSaveDoc(fileName, signedUrl, accessToken)


In [ ]:
# NLP package used to aid in text manipulation
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
# Machine Learning modules used to prepare and measure text
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
# HTML text processing
from bs4 import BeautifulSoup
# Helper modules
import matplotlib.pyplot as plt
from tqdm.notebook import trange  # Progress bar
# Refinitiv packages to extract filings data and retrieve price data
import refinitiv.data as rd
from refinitiv.data.content import filings
# Convenient modules to simplify API access to Filings
%run ./FilingsQuery.ipynb
%run ./SymbolLookup.ipynb
pd.set_option('display.max_colwidth', 60)

In [ ]:
# Open a session to retrieve Filings data from RDP
rd.open_session()


In [ ]:
# Load the models
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)

In [ ]:
# Download the Pre-trained transformer used to process our raw text
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')


In [ ]:
prediction = nlp(
    "The future for next years sales will increase by 10 %.", top_k=3)
prediction


In [ ]:
# Sentiment - Download the Pre-trained transformer used to process our raw text
sent_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")


In [ ]:
# Sentiment - Download the FinBert model used to process our transformed data
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
